# **Домашнее задание №2**

<!-- В данном задании вам предлагаются 3 задачи: -->

Вам предлагается использовать функцию потерь [ArcFace](https://arxiv.org/abs/1801.07698) для решения задачи Metric Learning в распознавании лиц.  
Подробное описание задачи вы можете найти в соответствующем разделе домашнего задания.  
**Ваша цель:** Обучить две модели для предсказания дискриминативных представлений изображений, $\textbf{z}\in\mathbb{R}^{2}$, с помощью функции потерь SoftMax и функции из статьи [ArcFace](https://arxiv.org/abs/1801.07698) (см. описание задачи 1). Модели необходимо обучить на 8-ми самых многочисленных классах из набора изображений лиц [MS1M-ArcFace](https://github.com/deepinsight/insightface/tree/master/recognition/_datasets_).  Затем нужно изобразить полученные представления на двух рисунках (один для SoftMax другой для ArcFace функций потерь).

Рисунок для SoftMax функции будет вам дан.  
**Вспомогательный код для загрузки данных и для обучения модели с помощью SoftMax функции потерь вы можете найти в разделе с заданием.**  
Помогает ли функция потерь ArcFace для улучшения дикриминативных способностей векторов представлений?


## **Задача - обучить небольшой ArcFace модели (1 балл)**

При решении задачи распознавания лиц возникает необходимость ответить на вопрос: изображен ли на двух разных картинках один и тот же человек или нет?  
На этот вопрос можно ответить с помощью функции расстояния между изображениями, учитывающую абстрактную семантическую информацию.  

Современные методы распознавания лиц используют большие наборы данных, содержащие изображения разных людей, для обучения нейросетей, вычисляющих "осмысленные" вектора представлений изображений лиц.
Для каждого изображения лица человека $\textbf{x}$ с помощью обучаемой функции $f_{\theta}(*)$ вычисляется вектор представления $\textbf{x}$:
$$
\textbf{z} = f_{\theta}(\textbf{x})
$$
Параметры $\theta$ модели подбираются так, чтобы расстояние между векторами представлений изображений разных людей было велико, а между представлениями изображений одного и того же человека - низко:
$$
d(\textbf{z}_i,\textbf{z}_j)>>d(\textbf{z}_i,\textbf{z}'_i)
$$
где $\mathbf{id}(\textbf{x}_i)=\mathbf{id}(\textbf{x}'_i),\,\mathbf{id}(\textbf{x}_i)\neq\mathbf{id}(\textbf{x}_j)$, $d$ -- некоторая функция расстояния, например, косинусное расстояние:
$$
d(\textbf{z}_i,\textbf{z}_j) = -\frac{\langle\textbf{z}_i,\textbf{z}_j\rangle}{\left\lVert\textbf{z}_i\right\rVert\left\lVert\textbf{z}_j\right\rVert}
$$

Для получения искомых представлений можно решать задачу классификации на большом наборе данных с изображениями разных людей, а затем использовать вектор перед последним линейным слоем как вектор представления лица.
В такой постановке каждый человек в наборе данных рассматривается как отдельный класс, а множество изображений его лица - как представители этого класса.

Для решения задачи многоклассовой классификации можно использовать стандартную Softmax функцию потерь:
$$
    L = -\frac{1}{N}\sum_{i=1}^N\log\frac{e^{W_{y_i}^T\textbf{z}_i + b_{y_i}}}{\sum_{j=1}^ne^{W_{j}^T\textbf{z}_i + b_{j}}}
$$
$\textbf{z}_i$ - представление изображения, $N$ - число изображений в мини батче, $W_j, b_j$ - параметры весов (гиперплоскости) для каждого класса.

Если мы отнормируем вектор $\textbf{x}$ так, чтобы он имел ограниченную норму $s$, положим норму вектора $W_j$ равную 1, а $b_j = 0$, то функция потерь может быть переписана в следующей форме:
$$
L = -\frac{1}{N}\sum_{i=1}^N
\log
\frac{e^{s\cos\theta_{y_i}}}
{e^{s\cos\theta_{y_i}} + \sum_{j\neq y_i}^ne^{s\cos\theta_{j}}},
$$
где $\theta_j$ - угол между представлением $i$-го изображения и вектором $W_j$ указывающим на центр $j$-ой класса.

Мы можем видеть, что здесь максимизируется косинус угла между представлением и соответствующим вектором класса.
На самом деле мы хотим минимизировать сам угол, а не максимизировать его косинус, потому что угол лучше соответствует близости в пространстве представлений.

Для решения этой задачи предлагается [ArcFace](https://arxiv.org/abs/1801.07698) функция потерь:
$$
L = -\frac{1}{N}\sum_{i=1}^N
\log
\frac{e^{s\cos(\theta_{y_i}+m)}}
{e^{s\cos(\theta_{y_i}+m)} + \sum_{j\neq y_i}^ne^{s\cos\theta_{j}}},
$$
где вводится параметр отступа $m=0.5$.

**Ваша цель:** Исследовать, как функция потерь влияет на качество получаемых представлений изображений лиц.

Для этого необходимо:

1. Обучить две глубокие модели для получения 2-мерных дискриминативных представлений изображений: первую - с помощью функции потерь SoftMax, а вторую - с помощью функции из статьи [ArcFace](https://arxiv.org/abs/1801.07698) (см. описание задачи 1). Для обучения предлагается использовать изображения 8-ми самых многочисленных классов из набора изображений лиц [MS1M-ArcFace](https://github.com/deepinsight/insightface/tree/master/recognition/_datasets_).
    В качестве модели предлагается использовать архитектуру iResNet50, код модели предоставлен в файле ```iresnet.py```. Вспомогательный код для загрузки данных, инициализации и обучения модели с помощью SoftMax функции потерь вы можете найти ниже. Вторую функцию потерь необходимо реализовать самостоятельно. **(0,25 балла)**
    
2. Оценить качество полученных двумерных представлений изображений визуально, изобразив их на окружности в $\mathbb{R}^2$. Визуализация представлений, полученных с помощью SoftMax функции приведена в ноутбуке ниже.**(0,25 балла)**
    
3. В качестве объективной метрики качества посчитать среднее расстояние от представлений изображений одного класса до луча центра класса. Помогает ли функция потерь ArcFace для улучшения дикриминативных способностей векторов представлений? **(0,25 балла)**

4. Изучить, как выбор гиперпараметров функции потерь ArcFace (значения нормы $s$ и отступа $m$) влияет на процесс обучения модели и качество полученных представлений. Удается ли получить хорошие представления при $m \neq 0.5$? **(0,25 балла)**

# **Бонусные задания:**

1. Решить ту же задачу для 3-мерного пространства представлений $\textbf{z}_{i}\in\mathbb{R}^{3}$, визуализировать полученные представления на шаре. **(0,25 балла)**

2. Реализовать [Triplet Loss](https://arxiv.org/abs/1503.03832) и обучить модель с его помощью. Сравнить полученные результаты. **(0,25 балла)**

Удачи!

### Загрузка данных

<!-- Загрузите выборку MS1M-ArcFace, и распакуйте данные в текущей дериктории:
https://drive.google.com/file/d/1SXS4-Am3bsKSK615qbYdbA_FMVh3sAvR/view. Если вы хотите сделать это напрямую в коде, можете воспользоваться командами, закомментированными в ячейке ниже. -->

In [18]:
!wget -nc https://dl.dropboxusercontent.com/s/a53udei76f9h270/ms1m_subset.zip
!unzip ms1m_subset.zip

--2023-07-10 17:38:55--  https://dl.dropboxusercontent.com/s/a53udei76f9h270/ms1m_subset.zip
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.70.15, 2620:100:6026:15::a27d:460f
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.70.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102620237 (98M) [application/zip]
Saving to: ‘ms1m_subset.zip’

ms1m_subset.zip     100%[===================>]  97.87M  22.5MB/s    in 4.7s    

2023-07-10 17:39:02 (20.6 MB/s) - ‘ms1m_subset.zip’ saved [102620237/102620237]

Archive:  ms1m_subset.zip
   creating: ms1m_subset/
  inflating: ms1m_subset/train.rec   
  inflating: ms1m_subset/train.idx   
  inflating: ms1m_subset/labels.npy  


Если вы работаете в Google Colab'е, то вам может потребоваться установить следующие библиотеки, которых там нет по умолчанию:

In [1]:
# !pip install pytorch_lightning
# !pip install mxnet

### Импортируем необходимые библиотеки

Обратите внимание, что мы используем модель iresnet50_normalized, которая определена в файле ```iresnet.py```. Для того чтобы ее импортировать, необходимо положить этот файл в одну папку с данным ноутбуком.

In [4]:
import numpy as np
import mxnet as mx

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

import pytorch_lightning as pl
from pytorch_lightning import Trainer

import math
import numbers
import os

from tqdm import tqdm
from pathlib import Path
from typing import Tuple, Optional, Dict

from iresnet import iresnet50_normalized

### Датасет

Ниже определен класс с необходимым нам датасетом изображений лиц. Обратите внимание, что он наследуется от класса ```torch.utils.data.Dataset```.

In [5]:
class ArcFaceDataset(Dataset):
    """
    ArcFace dataset loader,
    based on https://github.com/deepinsight/insightface/blob/master/recognition/arcface_torch/dataset.py
    """
    def __init__(self, root_dir: str, num_labels: int, test: bool = False) -> None:
        """Initialize ArcFace Dataset.

        :param root_dir: path to the folder containing face images
        :param num_labels: number of classes (people) to be used
        :param test: if True, create test dataset (no augmentations, no labels)
        """
        super(ArcFaceDataset, self).__init__()

        self.test = test

        # for test dataset, use default transformations (conver to torch.Tensor and normalize),
        # do not use augmentations
        if self.test:
            self.transform = transforms.Compose(
                [
                    transforms.ToPILImage(),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
                ]
            )

        # for train dataset, add augmentations of images (Random horizontal flip)
        else:
            self.transform = transforms.Compose(
                [
                    transforms.ToPILImage(),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
                ]
            )

        self.root_dir = root_dir
        path_imgrec = os.path.join(root_dir, "train.rec")
        path_imgidx = os.path.join(root_dir, "train.idx")

        # load pictures
        self.imgrec = mx.recordio.MXIndexedRecordIO(path_imgidx, path_imgrec, "r")
        s = self.imgrec.read_idx(0)
        header, _ = mx.recordio.unpack(s)
        
        self.imgidx = np.array(range(1, int(header.label[0])))

        # load or create labels
        labels_path = Path(root_dir) / "labels.npy"
        if labels_path.is_file():
            self.labels = np.load(labels_path)
        else:
            print('Listing labels...')
            labels = []
            for i in tqdm(range(len(self.imgidx))):
                idx = self.imgidx[i]
                s = self.imgrec.read_idx(idx)
                header, img = mx.recordio.unpack(s)
                label = header.label
                labels.append(int(label))
            self.labels = np.array(labels)
            # save labels
            np.save(labels_path, self.labels)

        unique_labels, unique_counts = np.unique(self.labels, return_counts=True)
        top_ids = np.argsort(unique_counts)[::-1][:num_labels]
        self.top_labels = unique_labels[top_ids]

        self.label_map = dict(
            zip(self.top_labels.tolist(), np.arange(len(self.top_labels)))
        )

    def __getitem__(self, index: int) -> Tuple[torch.Tensor, Optional[int]]:
        """Get item of a dataset.

        :param index: index of an item
        :return:
            - a tuple (image, label) for the train dataset
            - image for tht test dataset
        """
        idx = self.imgidx[index]
        s = self.imgrec.read_idx(idx)
        header, img = mx.recordio.unpack(s)
        label = header.label

        if not isinstance(label, numbers.Number):
            label = label[0]

        label = self.label_map[int(label)]
        label = torch.tensor(label, dtype=torch.long)
        sample = mx.image.imdecode(img).asnumpy()

        if self.transform is not None:
            sample = self.transform(sample)
        if self.test:
            return sample, label
        else:
            return sample, label

    def __len__(self) -> int:
        """Return size of the dataset."""
        return len(self.imgidx)

In [6]:
num_people = 4

path_to_data = "ms1m_subset"
dataset = ArcFaceDataset(path_to_data, num_labels=num_people)
people_ids = np.where(np.isin(dataset.labels, dataset.top_labels))[0]
people_set = torch.utils.data.Subset(dataset, people_ids)

In [7]:
print("Dataset size:", len(people_set))

Dataset size: 2342


### Модель

Ниже определен класс для используемое модели. Для удобства обучения мы работаем с библиотекой pytorhc_lightning и испольщуем класс ```pl.LightningModule```.

In [8]:
class MetricLearningModel(pl.LightningModule):
    """Lightning wrapper for a Metric Learning model."""
    def __init__(
        self, backbone: torch.nn.Module, loss: torch.nn.Module, num_labels: int
    ) -> None:
        """Initialize MetricLearningModel.

        :param backbone: core deef model to be trained
        :param loss: loss function to be used
        :param num_labels: number of target classes (people)
        """
        super().__init__()

        self.backbone = backbone
        self.loss = loss

        # parameters of the last linear layer initialized by the 'kaiming_uniform_'
        self.softmax_weights = torch.nn.Parameter(torch.empty((num_labels, 2)))
        torch.nn.init.kaiming_uniform_(self.softmax_weights, a=math.sqrt(5))

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """Forward pass through the model.

        :param x: batch of images
        :return a tuple of:
            - features: outputs of the backbone model a.k.a. embeddings
            - logits: result of the last linear transformations
        """
        backbone_outputs = self.backbone(x)
        features = backbone_outputs["feature"]
        norm_weights = F.normalize(self.softmax_weights, dim=1)
        logits = F.linear(features, norm_weights)
        return features, logits

    def training_step(self, batch: Tuple[torch.Tensor, torch.Tensor]) -> torch.Tensor:
        """Do a training step of the model.

        :param batch: batch of input images
        :return: value of the loss function
        """
        images, labels = batch
        features, logits = self(images)
        loss = self.loss(logits, labels)
        # log loss value
        self.log("train_loss", loss.item(), prog_bar=True)
        return loss

    def configure_optimizers(self) -> Dict[str, torch.optim.Optimizer]:
        params = list(self.parameters()) #+ [self.softmax_weights] #list(self.linear_norm.parameters())
        optimizer = torch.optim.AdamW(params, lr=1e-4, weight_decay=5e-5)
        return {
            "optimizer": optimizer,
        }

### Инициализация модели

В качестве backbone-модели предлагается использовать глубокую сверточную сеть iResNet50. С ее арзитектурой можно ознакомиться в модуле ```iresent.py```.
В первом эксперименте мы используем стандартную функцию потерь SoftMax, определенную в ```torch.nn```.
Для удобства визуализации мы используем пространство представлений размерности $2$ (```num_features=2```).

In [9]:
backbone_model = iresnet50_normalized(num_features=2)
softmax_loss = torch.nn.CrossEntropyLoss()

softmax_model = MetricLearningModel(backbone_model, softmax_loss, num_labels=num_people)

### Обучение модели

Определяем стандартные гиперпараметры и обучаем модель в течение 20 эпох.
Для обучения используем интерфейс ```pytorch_lightning```.

In [12]:
batch_size = 64
num_workers = 2
max_epochs = 20

In [13]:
# initialize trainer, use one GPU for training
trainer = Trainer(
    max_epochs=max_epochs,
    default_root_dir="outputs/softmax_train",
    accelerator="gpu",
    devices=1
)

# create train dataloader
train_dataloader = DataLoader(
    people_set,
    batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers,
)

# fit the model
trainer.fit(softmax_model, train_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A10') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name     | Type             | Params
----------------------------------------------
0 | backbone | IResNetNorm      | 30.8 M
1 | loss     | CrossEntropyLoss | 0     
----------------------------------------------
30.8 M    Trainable params
2         Non-trainable params
30.8 M    Total params
123.178   Total estimated model params size (MB)


Epoch 4: 100%|██████████| 36/36 [00:36<00:00,  1.02s/it, v_num=36, train_loss=0.805]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 36/36 [00:37<00:00,  1.04s/it, v_num=36, train_loss=0.805]


### Получение представлений

Ниже приведена функция для получения представлений изображений из ```test_dataloader```.

In [14]:
def predict_features(
    model: pl.LightningModule, test_dataloader: DataLoader, device: str = "cuda"
) -> Tuple[np.array, np.array]:
    """Transform images and get their embeddings.

    :param model: trained MetricLearningModel
    :param test_dataloader: DataLoader with images to be transformed
    :param device: 'gpu' or 'cuda', if available
    :return a tuple of:
        - numpy array with obtained features
        - true image labels (people id's)
    """
    model.to(device)

    # switch model to 'eval' mode: disable randomness, dropout, etc.
    model.eval()

    predicted_features = []
    image_labels = []

    for images, labels in tqdm(test_dataloader):
        images = images.to(device)
        features, _ = model(images)
        features = features.detach().cpu().numpy()
        labels = labels.numpy()
        predicted_features.append(features)
        image_labels.append(labels)

    predicted_features = np.concatenate(predicted_features)
    image_labels = np.concatenate(image_labels)

    return predicted_features, image_labels

In [15]:
num_people = 4

dataset = ArcFaceDataset(path_to_data, num_labels=num_people, test=True)
people_ids = np.where(np.isin(dataset.labels, dataset.top_labels))[0]
people_set = torch.utils.data.Subset(dataset, people_ids)

test_dataloader = DataLoader(
    people_set,
    batch_size=16,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers,
)

predicted_features, image_labels = predict_features(softmax_model, test_dataloader)

100%|██████████| 147/147 [00:14<00:00, 10.21it/s]


### Визуализация 2-мерных представлений

In [16]:
colors = list(mcolors.TABLEAU_COLORS)[:num_people]

softmax_weights = softmax_model.softmax_weights.detach().cpu()
softmax_weights = F.normalize(softmax_weights, dim=1).numpy()

In [ ]:
plt.figure(figsize=(6, 6))
for i, (weight, color) in enumerate(zip(softmax_weights, colors)):
    points = predicted_features[image_labels == i]
    x, y = [0, weight[0]], [0, weight[1]]
    plt.plot(x, y, marker="", c=color)
    plt.scatter(points[:, 0], points[:, 1], color=color, s=3)
    if i == (num_people - 1):
        break
plt.gca().set_aspect("equal")
plt.axis("off")
plt.show()

## Решение

In [ ]:
# место для вашего кода

## Выводы